In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading Data

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
 
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean.csv', skiprows = range(1,2868_000))
#test = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/test_clean.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   BROWSE_NODE_ID  898 non-null    int64 
 1   TEXT            898 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.2+ KB


In [ ]:
#test.info()

In [ ]:
train.isnull().sum()

BROWSE_NODE_ID    0
TEXT              0
dtype: int64

In [ ]:
#test.isnull().sum()

# Converting Browse node id

In [ ]:
train['BROWSE_NODE_ID'].value_counts()

In [ ]:
f = open('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/other_node_id.txt', 'r')
 
other_node_id = [int(i) for i in f.readline().split(',')[:-1]]
other_node_id = np.array(other_node_id)
 
f.close()
print(len(other_node_id))

7160


In [ ]:
train['BROWSE_NODE_ID'] = train['BROWSE_NODE_ID'].apply(lambda x: x if x not in other_node_id else -1)

In [ ]:
train['BROWSE_NODE_ID'].value_counts()

 1045     106
 5         33
-1         25
 811       24
 739       21
         ... 
 535        1
 1551       1
 5621       1
 27143      1
 451        1
Name: BROWSE_NODE_ID, Length: 350, dtype: int64

# Groupby Browse node id

In [ ]:
node_id_group = train.groupby('BROWSE_NODE_ID')

In [ ]:
df = node_id_group.get_group(5)
df.head(3)

# Keep only Top 50 words by class in Corpus

In [ ]:
def build_vocab(sentences):
  word_freq = {}
  for row in sentences:
      for word in row.split():
        if len(word)>2:
          try:
            word_freq[word] += 1
          except:
            word_freq[word] = 1
  
  #print("Total words found:",len(word_freq))
  return (word_freq, len(word_freq))

'def build_vocab(sentences):\n  word_freq = {}\n  for row in sentences:\n      for word in row.split():\n        if len(word)>2:\n          try:\n            word_freq[word] += 1\n          except:\n            word_freq[word] = 1\n  \n  #print("Total words found:",len(word_freq))\n  return (word_freq, len(word_freq))'

In [ ]:
vocab,leng = build_vocab(df['TEXT'])

"vocab,leng = build_vocab(df['TEXT'])"

In [ ]:
sorted_words_freq = sorted(vocab.items(), key = lambda x: x[1], reverse = True)
sorted_words_freq[:50]

'sorted_words_freq = sorted(vocab.items(), key = lambda x: x[1], reverse = True)\nsorted_words_freq[:40]'

In [ ]:
word = []
for i in sorted_words_freq[:50]:
  word.append(i[0])

'word = []\nfor i in sorted_words_freq[:40]:\n  word.append(i[0])'

In [ ]:
len(word)

'len(word)'

In [ ]:
corpus = {}
ctr = 1
total_words_processed = 0
for name,group in tqdm(node_id_group):
  vocab, length = build_vocab(group['TEXT'])
  sorted_words_freq = sorted(vocab.items(), key = lambda x: x[1], reverse = True)
  total_words_processed+=length
  
  word = {}
  for i in sorted_words_freq[:50]:
    word[i[0]] = 0
  
  corpus.update(word)
  print('Total words processed:',total_words_processed, 'Total unque words in the corpus:', len(corpus))
  print(round((ctr/2760)*100,2),'% completed:')
  ctr+=1

 
print(len(corpus))

"corpus = {}\nctr = 1\ntotal_words_processed = 0\nfor name,group in tqdm(node_id_group):\n  vocab, length = build_vocab(group['TEXT'])\n  sorted_words_freq = sorted(vocab.items(), key = lambda x: x[1], reverse = True)\n  total_words_processed+=length\n  \n  word = {}\n  for i in sorted_words_freq[:50]:\n    word[i[0]] = 0\n  \n  corpus.update(word)\n  print('Total words processed:',total_words_processed, 'Total unque words in the corpus:', len(corpus))\n  print(round((ctr/2760)*100,2),'% completed:')\n  ctr+=1\n\n \nprint(len(corpus))"

In [ ]:
print(len(corpus))

'print(len(corpus))'

In [ ]:
corpus = set(corpus)
print(len(corpus))

'corpus = set(corpus)\nprint(len(corpus))'

In [ ]:
with open('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/corpus.txt','a') as f:
  for word in corpus:
    f.write(f'{word},')

"with open('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/corpus.txt','a') as f:\n  for word in corpus:\n    f.write(f'{word},')"

In [ ]:
with open('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/corpus.txt','r') as f:
  corpus = f.readline().split(',')[:-1]
  print(len(corpus))

"with open('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/corpus.txt','r') as f:\n  corpus = f.readline().split(',')[:-1]\n  print(len(corpus))"

# 

In [ ]:
corpus[:10]

'corpus[:10]'

# Keeping only corpus words in train and test

In [ ]:
with open('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/corpus.txt','r') as f:
  corpus = f.readline().split(',')[:-1]
  print(len(corpus))

8138


In [ ]:
corpus = np.array(corpus)
print(len(corpus))
print(corpus)

8138
['hardness' 'bulletin' 'splitter' ... 'corrosion' 'tie' 'newsboy']


In [ ]:
def clean(text):
  cleantext = ''
  for word in text.split():
    if word in corpus:
      cleantext = cleantext + ' ' + word
 
  return cleantext.strip()

In [ ]:
clean('pete cat bedtime blue 56 doll inch pete cat coole')

'cat blue doll inch cat'

In [ ]:
FROM = 0
TO = 897

In [ ]:
temp = train.loc[FROM:TO,:]
temp.reset_index(drop=True, inplace=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   BROWSE_NODE_ID  898 non-null    int64 
 1   TEXT            898 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.2+ KB


In [ ]:
temp.head(2)

,BROWSE_NODE_ID,TEXT
0,1209,sarona men olive slim fit stretch jogger jean ...
1,1045,pikkme samsung galaxy back cover case designer...


In [ ]:
batch_size = 200
ctr = batch_size
leng = len(temp)
 
for start,end in zip(range(0,leng+1,batch_size), range(batch_size,leng+1,batch_size)):
  
  print('start:',start,'end:',end)
  
  df = temp.loc[start:end,:]
  df["CLEANED TEXT"] = df["TEXT"].apply(clean)
  df.drop('TEXT', axis=1, inplace=True)
  
  if not os.path.isfile('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/train_clean_final.csv'):
    df.to_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/train_clean_final.csv', index = False)
  else:
    df.to_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/train_clean_final.csv', mode = 'a', header = False, index = False)
  
  print(round((ctr/leng)*100, 2), '% completed')
  ctr+=batch_size

start: 0 end: 200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


22.27 % completed
start: 200 end: 400
44.54 % completed
start: 400 end: 600
66.82 % completed
start: 600 end: 800
89.09 % completed


In [ ]:
temp1 = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/train_clean_final.csv')
temp1.head(3)

,BROWSE_NODE_ID,CLEANED TEXT
0,0,cat blue doll inch cat popular cat new cat blu...
1,1,new refrigerator magnet new cello wrapped hard...
2,-1,ultimate self complete guide baking craft gard...


In [ ]:
temp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3167585 entries, 0 to 3167584
Data columns (total 2 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   BROWSE_NODE_ID  int64 
 1   CLEANED TEXT    object
dtypes: int64(1), object(1)
memory usage: 48.3+ MB


In [ ]:
temp1.isnull().sum()

BROWSE_NODE_ID       0
CLEANED TEXT      9159
dtype: int64

In [ ]:
temp1.tail(3)

,BROWSE_NODE_ID,CLEANED TEXT
3167582,5,unique store limited edition shirt men fresh c...
3167583,1052,thought solid woven cotton silk saree woman st...
3167584,1209,men olive slim fit stretch jogger jean refresh...


In [ ]:
temp1.duplicated().sum()

835469

In [ ]:
del(temp)

In [ ]:
temp.tail(3)

,BROWSE_NODE_ID,CLEANED TEXT
100047,374,hardness selfie note tempered glass screen gua...
100048,1045,pug dog printed designer hard back case cover ...
100049,1045,movie direction board printed designer hard ba...


In [ ]:
train.loc[99995:100000,:]

,BROWSE_NODE_ID,TEXT
99995,2667,hokipo antislip rubber back dirt trapper doorm...
99996,924,zivame woman pyjama set pfashapink pink medium...
99997,368,exotic india men oxblood red leather mojaris s...
99998,374,fasheen hardness micromax selfie note tempered...
99999,1045,nalamicases pug dog printed designer hard back...
100000,1045,srrlcases movie direction clap board icon prin...


In [ ]:
#train.drop('TEXT', axis=1, inplace=True)
#train.head()

In [ ]:
#train.isnull().sum()

# Exporting

In [ ]:
#train.to_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean_final.csv', index = False)

# END